# 6、将你的其中一个假设落实，如果能完成两个更好，但不强求

利用TimingSignal最后输出long、short、long_short这三个绩效表出来


In [39]:
import warnings
from jaqs_fxdayu.data.dataservice import LocalDataService
from jaqs_fxdayu.data.hf_dataview import HFDataView
import pandas as pd
warnings.filterwarnings('ignore')
ds = LocalDataService()

## 加freq参数
path = r'../min_data/VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume',
         'symbol': 'BTCUSDT:binance',
         'freq': '30Min,4H',
         'start_date':20180601000000}

Time_dict = ds.bar_reader(path,props) #读取数据

dv4H = HFDataView()
dv4H.create_init_dv(Time_dict['4H'].dropna().set_index(["trade_date","symbol"]))

dv4H.add_formula('MACD',"Ta('EMA',0,0,0,0,close,0,12)-Ta('EMA',0,0,0,0,close,0,26)",add_data=True)
dv4H.add_formula('MACD_Signal',"Ta('EMA',0,0,0,0,MACD,0,9)",add_data=True)
dv4H.add_formula('MACD_Hist',"(MACD-MACD_Signal)",add_data=True)

MACD=dv4H.get_ts('MACD', date_type='datetime') 
MACD_Signal=dv4H.get_ts('MACD_Signal', date_type='datetime') 
MACD_Hist=dv4H.get_ts('MACD_Hist', date_type='datetime') 

Initialize dataview success.


In [41]:
#验证假设：MACD因子由小于0变为大于0进场，由大于0变为小于0出场
from jaqs_fxdayu.data.hf_dataview import HFDataView
#进场信号
long = dv4H.add_formula('longMACD','If((MACD_Hist>0) && (Delay(MACD_Hist,1)<=0),2,0)',add_data = True)    
short = dv4H.add_formula('shortMACD','If((MACD_Hist<=0) && (Delay(MACD_Hist,1)>0),-2,0)',add_data = True)  
#出场信号
close_long = dv4H.add_formula('closeLongMACD','If(longMACD==-2,1,0)',add_data = True)
close_short = dv4H.add_formula('closeShortMACD','If(shortMACD==2,-1,0)',add_data = True)

dv4H.get_ts('longMACD').stack()[dv4H.get_ts('longMACD').stack()==2]
dv4H.get_ts('shortMACD').stack()[dv4H.get_ts('shortMACD').stack()==2]

Field [longMACD] is overwritten.
Field [shortMACD] is overwritten.
Field [closeLongMACD] is overwritten.
Field [closeShortMACD] is overwritten.


Series([], dtype: float64)

In [43]:
from jaqs_fxdayu.research import TimingDigger

def TimingSignal(td, dv, long='long', short='short', closeLong='closeLong', closeShort='closeShort', mhp=None, sl=None, sp=None):
    
    td.process_signal(
        sig_type='long',
        enter_signal=dv.get_ts(long),
        exit_signal=dv.get_ts(closeLong),
        price=dv.get_ts('close'),
        max_holding_period=mhp,
        stoploss=-sl,
        stopprofit=sp
    )
    
    td.process_signal(
        sig_type='short',
        enter_signal=dv.get_ts(short),
        exit_signal=dv.get_ts(closeShort),
        price=dv.get_ts('close'),
        max_holding_period=mhp,
        stoploss=-sl,
        stopprofit=sp
    )
tdMACD = TimingDigger(output_folder='.', output_format='pdf', signal_name='MACD_Hist')
TimingSignal(tdMACD, dv4H, 'longMACD', 'shortMACD', 'closeLongMACD', 'closeShortMACD', mhp=100, sl=0.05, sp=0.3)

tdMACD.create_event_report(sig_type="long")
tdMACD.create_event_report(sig_type="short")
tdMACD.create_event_report(sig_type="long_short")

Nan Data Count (should be zero) : 0;  Percentage of effective data: 3%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 3%
*****-Summary-*****
Event Analysis
                      win    loss     all
t-stat              2.293 -10.255  -0.362
p-value             0.084   0.000   0.722
mean                0.122  -0.059  -0.009
std                 0.106   0.020   0.100
info_ratio          1.147  -2.960  -0.088
skewness            0.968   1.428   2.049
kurtosis           -0.384   3.366   3.732
pct5                0.018  -0.084  -0.081
pct25               0.055  -0.064  -0.063
pct50               0.096  -0.060  -0.055
pct75               0.130  -0.055   0.006
pct95               0.280  -0.032   0.158
occurance           5.000  13.000  18.000
win_ratio             NaN     NaN   0.278
win_mean/loss_mean    NaN     NaN   2.065
*****-Summary-*****
Event Analysis
                      win    loss     all
t-stat              4.572  -7.591   0.212
p-value             0.004   0.00

In [44]:
path = r'../min_data/VnTrader_1Min_Db'
props = {'fields': 'open,high,low,close,volume',
         'symbol': 'EOSUSDT:binance',
         'freq': '30Min,2H',
         'start_date':20180601000000}

Time_dict = ds.bar_reader(path,props) #读取数据

from jaqs_fxdayu.data.hf_dataview import HFDataView

dv2H = HFDataView()
dv2H.create_init_dv(Time_dict['2H'].dropna().set_index(["trade_date","symbol"]))

dv2H.add_formula('MACD',"Ta('EMA',0,0,0,0,close,0,12)-Ta('EMA',0,0,0,0,close,0,26)",add_data=True)
dv2H.add_formula('MACD_Signal',"Ta('EMA',0,0,0,0,MACD,0,9)",add_data=True)
dv2H.add_formula('MACD_Hist',"(MACD-MACD_Signal)",add_data=True)

MACD=dv2H.get_ts('MACD', date_type='datetime') 
MACD_Signal=dv2H.get_ts('MACD_Signal', date_type='datetime') 
MACD_Hist=dv2H.get_ts('MACD_Hist', date_type='datetime') 

Initialize dataview success.


In [46]:
#进场信号
long = dv2H.add_formula('longMACD','If((MACD_Hist>0) && (Delay(MACD_Hist,1)<=0),2,0)',add_data = True)    
short = dv2H.add_formula('shortMACD','If((MACD_Hist<=0) && (Delay(MACD_Hist,1)>0),-2,0)',add_data = True)  
#出场信号
close_long = dv2H.add_formula('closeLongMACD','If(longMACD==-2,1,0)',add_data = True)
close_short = dv2H.add_formula('closeShortMACD','If(shortMACD==2,-1,0)',add_data = True)

dv2H.get_ts('longMACD').stack()[dv2H.get_ts('longMACD').stack()==2]
dv2H.get_ts('shortMACD').stack()[dv2H.get_ts('shortMACD').stack()==2]

Series([], dtype: float64)

In [47]:
from jaqs_fxdayu.research import TimingDigger

def TimingSignal(td, dv, long='long', short='short', closeLong='closeLong', closeShort='closeShort', mhp=None, sl=None, sp=None):
    
    td.process_signal(
        sig_type='long',
        enter_signal=dv.get_ts(long),
        exit_signal=dv.get_ts(closeLong),
        price=dv.get_ts('close'),
        max_holding_period=mhp,
        stoploss=-sl,
        stopprofit=sp
    )
    
    td.process_signal(
        sig_type='short',
        enter_signal=dv.get_ts(short),
        exit_signal=dv.get_ts(closeShort),
        price=dv.get_ts('close'),
        max_holding_period=mhp,
        stoploss=-sl,
        stopprofit=sp
    )
tdMACD = TimingDigger(output_folder='.', output_format='pdf', signal_name='MACD_Hist')
TimingSignal(tdMACD, dv2H, 'longMACD', 'shortMACD', 'closeLongMACD', 'closeShortMACD', mhp=100, sl=0.05, sp=0.3)

tdMACD.create_event_report(sig_type="long")
tdMACD.create_event_report(sig_type="short")
tdMACD.create_event_report(sig_type="long_short")

Nan Data Count (should be zero) : 0;  Percentage of effective data: 3%
Nan Data Count (should be zero) : 0;  Percentage of effective data: 3%
*****-Summary-*****
Event Analysis
                      win    loss     all
t-stat              3.772 -21.346  -1.370
p-value             0.005   0.000   0.178
mean                0.171  -0.070  -0.023
std                 0.129   0.020   0.113
info_ratio          1.334  -3.558  -0.204
skewness           -0.038  -0.480   2.207
kurtosis           -1.777  -0.344   3.619
pct5                0.011  -0.108  -0.106
pct25               0.040  -0.081  -0.076
pct50               0.159  -0.067  -0.062
pct75               0.305  -0.054  -0.051
pct95               0.313  -0.051   0.304
occurance           9.000  37.000  46.000
win_ratio             NaN     NaN   0.196
win_mean/loss_mean    NaN     NaN   2.436
*****-Summary-*****
Event Analysis
                       win    loss     all
t-stat              11.180 -18.947   1.813
p-value              0.000   0